In [1]:
# google: torchvision optical flow

from pathlib import Path
import numpy as np
from skimage import io
from skimage.transform import rescale, resize
from scipy.ndimage import gaussian_filter
from scipy import ndimage

import torch
import matplotlib.pyplot as plt
import torchvision.transforms.functional as F
import torchvision.transforms as T

imginfo = lambda img: print(type(img), img.dtype, img.shape, img.min(), img.max())

/home/iternal/miniconda3/envs/rerun/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/iternal/miniconda3/envs/rerun/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
from torchvision.models.optical_flow import raft_large

device = "cpu"
model = raft_large(pretrained=True, progress=False).to(device)
model = model.eval()

/home/iternal/miniconda3/envs/rerun/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/iternal/miniconda3/envs/rerun/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Raft_Large_Weights.C_T_SKHT_V2`. You can also use `weights=Raft_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
# === load first touch coors
import json
with open("data/single_log.json") as f:
    episode_log = json.load(f)
first_x, first_y, first_z = map(int, episode_log["first_touch"])
cropw = 200

def cut_crop(image):
    """
    :param image: [h, w, c]
    """
    return image[first_y-cropw:first_y+cropw, first_x-cropw:first_x+cropw]

In [22]:
# === Load images
file_list = sorted(Path("data/frames").glob("center*jpg"))
print("image list:", len(file_list))

before = io.imread(file_list[0]).astype(np.float32)
grip = io.imread("data/frames/grip_image.jpg").astype(np.float32)
after = io.imread(file_list[5]).astype(np.float32) # [h, w, c]

#before = cut_crop(before)
#grip = cut_crop(grip)
#after = cut_crop(after)

io.imsave("data/crop_1.jpg", before.astype(np.uint8))
io.imsave("data/crop_2.jpg", grip.astype(np.uint8))
io.imsave("data/crop_3.jpg", after.astype(np.uint8))

image list: 8


In [23]:
def to_batch(image):
    batch = torch.from_numpy(image)[None, :, :, :].to(torch.float32) / 255  # [h, w, c] -> [1, h, w, c]
    batch = batch.permute(0, 3, 1, 2)  # [1, h, w, c] -> [1, c, h, w]
    
    transforms = T.Compose(
        [
            T.ConvertImageDtype(torch.float32),
            T.Normalize(mean=0.5, std=0.5),  # map [0, 1] into [-1, 1]
            #T.Resize(size=(520, 960)),
        ]
    )
    batch = transforms(batch)
    return batch

img1_batch = to_batch(grip)
img2_batch = to_batch(after)
imginfo(img1_batch)

<class 'torch.Tensor'> torch.float32 torch.Size([1, 3, 720, 1280]) tensor(-1.) tensor(1.)


In [24]:
from torchvision.utils import flow_to_image

def compute_flow(img1_batch, img2_batch):
    with torch.no_grad():
        list_of_flows = model(img1_batch.to(device), img2_batch.to(device))
    print(f"length = {len(list_of_flows)} = number of iterations of the model")
    predicted_flows = list_of_flows[-1].detach()

    flow_imgs = flow_to_image(predicted_flows)
    
    # The images have been mapped into [-1, 1] but for plotting we want them in [0, 1]
    img1_batch = [(img1 + 1) / 2 for img1 in img1_batch]
    
    grid = [[img1, flow_img] for (img1, flow_img) in zip(img1_batch, flow_imgs)]
    
    return grid[0][1]
    

In [25]:
flow = compute_flow(img1_batch, img2_batch)

def save_torch(path, tensor):
    """
    tensor: [c, h, w] cuda float32 [0, 1]
    """
    img = (tensor.to("cpu").permute([1, 2, 0]).numpy() * 255).astype(np.uint8)
    io.imsave(path, img)

save_torch("data/raft_flow.jpg", flow)

length = 12 = number of iterations of the model


In [26]:
img1_batch = to_batch(grip)
img2_batch = to_batch(before)
flow = compute_flow(img1_batch, img2_batch)
save_torch("data/raft_flow_2.jpg", flow)

length = 12 = number of iterations of the model
